# Wholesale customers Data Set (卸売業者の顧客データ)
https://archive.ics.uci.edu/ml/datasets/wholesale+customers <br>
ポルトガルの卸売業者の顧客のデータ（2011年(通年)、通貨単位の年間支出）

In [ ]:
# -*- coding: utf-8 -*-
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
from matplotlib import cm
import seaborn as sns

from sklearn.cluster import KMeans
from sklearn import datasets
#from sklearn.metrics import  accuracy_score, classification_report

FLAG_fig = False

In [ ]:
#原データは　https://archive.ics.uci.edu/ml/datasets/wholesale+customers#　にあり，
#このファイル名の空白を下線に変えて，下記のサイトにアップした。
df_all = pd.read_csv("https://sites.google.com/site/datasciencehiro/datasets/Wholesale_customers_data.csv")
df_all.head()

In [ ]:
#不要なカラムを削除
df_1 = df_all.drop(['Channel', 'Region'], axis=1)
df_1.head()

In [ ]:
# pandasデータフレームをnumpy配列に変換
# .tolist()は"値"をリスト（Python形式）に変換する
cstmr_data = np.array([df_1['Fresh'].values,
                       df_1['Milk'].values,
                       df_1['Grocery'].values,
                       df_1['Frozen'].values,
                       df_1['Detergents_Paper'].values,
                       df_1['Delicassen'].values
                       ])
cstmr_data = cstmr_data.T # Transpose, 転置

In [ ]:
print(type(cstmr_data))
cstmr_data[0:5] # numpy 配列の中身の確認

In [ ]:
clstr = KMeans(n_clusters=4).fit_predict(cstmr_data)
clstr

クラスタリングの結果をpandasのデータフレームに挿入する。

In [ ]:
df_1['cluster_id'] = clstr
df_1.head()

In [ ]:
# 各クラスタのサンプル数の分布
df_1['cluster_id'].value_counts()

In [ ]:
# クラスタ==0 商品の支出額の平均値
df_1[df_1['cluster_id']==0].mean() 

In [ ]:
# クラスタ==1 商品の支出額の平均値
df_1[df_1['cluster_id']==1].mean() 

In [ ]:
# クラスタ==2 商品の支出額の平均値
df_1[df_1['cluster_id']==2].mean() 

In [ ]:
# クラスタ==3 商品の支出額の平均値
df_1[df_1['cluster_id']==3].mean() 

In [ ]:
# クラスタ毎のグルーピング
df2 = df_1.groupby('cluster_id').mean()
df2

pandasのplot.bar https://pandas.pydata.org/pandas-docs/stable/generated/pandas.DataFrame.plot.bar.html<br>
この引数の詳細　https://pandas.pydata.org/pandas-docs/stable/generated/pandas.DataFrame.plot.html

In [ ]:
df2.plot.bar( alpha=0.6, figsize=(6,4), stacked=True, cmap='Set1')
if FLAG_fig: plt.savefig('fig_kMeans_WholeSale_Bar.png')
plt.show()

結果の考察

クラスタ番号 = 0 顧客 ( 79件) 、Fresh (生鮮品) やFrozen (冷凍食品) の支出額が比較的高い<br>
クラスタ番号 = 1 顧客 (  7件) 、全てのジャンルで支出額が高い<br>
クラスタ番号 = 2 顧客 ( 77件) 、Milk，Grocery，Detergents_Paperの支出額が比較的高い<br>
クラスタ番号 = 3 顧客 (280件) 、全体的に支出額が低い傾向

In [ ]:
#pd.tools.plotting.scatter_matrix(df_1)
g = sns.pairplot(df_1)

if FLAG_fig: plt.savefig('fig_kMeans_WholeSale_Pairplot.png')
plt.show()

上記の結果で，Grocery，Detergents_Paperの相関が強く，かつ支出額が比較的高いことに注目して，この2変数だけでのクラスタリングを実施する

In [ ]:
cstmr_data = np.array([df_1['Grocery'].tolist(),
                       df_1['Detergents_Paper'].tolist()
                       ])
cstmr_data = cstmr_data.T
clstr = KMeans(n_clusters=3).fit_predict(cstmr_data)
clstr

この結果を用いた散布図を描く。何らかのクラスタが認められる

In [ ]:
plt.scatter(cstmr_data[:,0], cstmr_data[:,1], c=clstr, cmap=cm.bwr, edgecolors='k')
plt.xlabel('Grocery')
plt.ylabel('Detergents_Paper')

if FLAG_fig: plt.savefig('fig_kMeans_WholeSale_Scatter.png')
plt.show()